In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[4],3b
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'co2': 0.0004, 'h2o': 'atmpro'}","{'h2o': 'atmpro', 'co2': 0.0004}"
nv,-,1000
tsfc,294,294


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312        1  76.716477                    1  54.493314
0.000750        2  40.975266                    2  41.525733
0.001052        3  33.041806                    3  35.820506
0.001476        4  26.177273                    4  29.287696
0.002070        5  20.453729                    5  22.421178
0.002904        6  15.887172                    6  16.168235
0.004074        7  12.355622                    7  11.340237
0.005714        8   9.670589                    8   8.313615
0.008015        9   7.632345                    9   6.630981
0.011243       10   6.059436                   10   5.575758
0.015771       11   4.801153                   11   4.583701
0.022122       12   3.733531                   12   3.497665
0.031031       13   2.741821                   13   2.366122
0.043528       14   1.535444                   14   1.084627
0.061057       15   0.621692                   15   0.254852
0.085645       16   1.009796                   16   0.809309
0.120136       17   1.979216                   17   1.948677
0.168516       18   2.674290                   18   2.799230
0.236378       19   3.354145                   19   3.532679
0.331549       20   4.138402                   20   4.173018
0.465100       21   5.134848                   21   4.934948
0.652400       22   6.676295                   22   6.401765
0.915100       23   8.422255                   23   8.267836
1.283650       24   8.121785                   24   8.030213
1.800600       25   6.305420                   25   6.110901
2.525700       26   4.920157                   26   4.624483
3.542800       27   3.938123                   27   3.887147
4.969550       28   3.204642                   28   3.339312
6.970850       29   2.670490                   29   2.764486
9.778100       30   2.245281                   30   2.258388
13.715850      31   1.823620                   31   1.848575
19.239350      32   1.385848                   32   1.350316
26.987250      33   1.051379                   33   1.013477
37.855300      34   0.824175                   34   0.878242
53.100050      35   0.576412                   35   0.632283
73.887500      36   0.318304                   36   0.302683
97.662500      37   0.177325                   37   0.150928
121.437500     38   0.132179                   38   0.137823
145.212500     39   0.033080                   39   0.057984
168.987500     40  -0.099825                   40  -0.075233
192.762500     41  -0.095180                   41  -0.079988
216.537500     42   0.024981                   42   0.027475
240.312500     43   0.077872                   43   0.071295
264.087500     44   0.094879                   44   0.084381
287.862500     45   0.101746                   45   0.090256
311.637500     46   0.103508                   46   0.090070
335.412500     47   0.102678                   47   0.090108
359.187500     48   0.100513                   48   0.088066
382.962500     49   0.097735                   49   0.086568
406.737500     50   0.094659                   50   0.085677
430.512500     51   0.090941                   51   0.084531
454.287500     52   0.085924                   52   0.080836
478.062500     53   0.079580                   53   0.076757
501.837500     54   0.072646                   54   0.071036
525.612500     55   0.066034                   55   0.065491
549.387500     56   0.060212                   56   0.060037
573.162500     57   0.055206                   57   0.054914
596.937500     58   0.051038                   58   0.050263
620.712500     59   0.047189                   59   0.045767
644.487500     60   0.043572                   60   0.041506
668.262500     61   0.040239                   61   0.037652
692.037500     62   0.037272                   62   0.033237
715.812500     63   0.034819                   63

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -17.738220   0.000000 -17.738220 -17.431136 -1.304198e-07   
0.000624    2     -17.738562   0.006016 -17.732546 -17.431512  4.406713e-03   
0.000876    3     -17.738717   0.007392 -17.731325 -17.431669  5.801454e-03   
0.001229    4     -17.738915   0.008970 -17.729945 -17.431873  7.502381e-03   
0.001723    5     -17.739163   0.010752 -17.728410 -17.432136  9.481940e-03   
0.002417    6     -17.739476   0.012748 -17.726729 -17.432476  1.166527e-02   
0.003391    7     -17.739876   0.014979 -17.724897 -17.432917  1.397065e-02   
0.004757    8     -17.740389   0.017491 -17.722898 -17.433493  1.638208e-02   
0.006672    9     -17.741055   0.020352 -17.720704 -17.434251  1.902646e-02   
0.009359    10    -17.741933   0.023659 -17.718275 -17.435259  2.214536e-02   
0.013128    11    -17.743108   0.027539 -17.715569 -17.436613  2.598859e-02   
0.018415    12    -17.744713   0.032150 -17.712563 -17.438464  3.071045e-02   
0.025830    13    -17.746962   0.037679 -17.709283 -17.441050  3.636908e-02   
0.036232    14    -17.750245   0.044340 -17.705904 -17.444778  4.301321e-02   
0.050823    15    -17.755665   0.052414 -17.703251 -17.450743  5.085339e-02   
0.071291    16    -17.764995   0.063252 -17.701743 -17.460642  6.137017e-02   
0.100000    17    -17.778772   0.080463 -17.698309 -17.475008  7.848797e-02   
0.140271    18    -17.796685   0.107817 -17.688868 -17.493521  1.062987e-01   
0.196760    19    -17.819499   0.148526 -17.670972 -17.516796  1.483060e-01   
0.275997    20    -17.847890   0.208400 -17.639490 -17.545272  2.099437e-01   
0.387100    21    -17.881570   0.296545 -17.585025 -17.578437  2.980359e-01   
0.543100    22    -17.917160   0.427024 -17.490136 -17.612818  4.236206e-01   
0.761700    23    -17.937519   0.620264 -17.317255 -17.631801  6.083922e-01   
1.068500    24    -17.900548   0.889380 -17.011168 -17.595024  8.721220e-01   
1.498800    25    -17.793142   1.195958 -16.597184 -17.491328  1.177785e+00   
2.102400    26    -17.646393   1.500051 -16.146342 -17.349595  1.473031e+00   
2.949000    27    -17.470616   1.817696 -15.652920 -17.179078  1.766332e+00   
4.136600    28    -17.267239   2.168333 -15.098906 -16.979354  2.113507e+00   
5.802500    29    -17.038372   2.571863 -14.466509 -16.753256  2.546449e+00   
8.139200    30    -16.780484   3.053163 -13.727321 -16.495213  3.053692e+00   
11.417000   31    -16.484836   3.629309 -12.855526 -16.196064  3.631518e+00   
16.014700   32    -16.157769   4.295441 -11.862327 -15.861762  4.304110e+00   
22.464000   33    -15.841878   5.038291 -10.803587 -15.539116  5.013165e+00   
31.510500   34    -15.559863   5.882957  -9.676906 -15.250644  5.810870e+00   
44.200100   35    -15.278108   6.840081  -8.438027 -14.963920  6.844433e+00   
62.000000   36    -15.025457   7.802808  -7.222649 -14.708741  7.922577e+00   
85.775000   37    -14.952916   8.626712  -6.326204 -14.637352  8.703729e+00   
109.550000  38    -15.053452   9.226652  -5.826801 -14.746036  9.237517e+00   
133.325000  39    -15.216914   9.762371  -5.454543 -14.921638  9.801313e+00   
157.100000  40    -15.556524  10.195144  -5.361380 -15.278069  1.032106e+01   
180.875000  41    -16.302836  10.660318  -5.642518 -16.050455  1.088155e+01   
204.650000  42    -17.388686  11.478110  -5.910576 -17.165968  1.177177e+01   
228.425000  43    -18.534089  12.693867  -5.840222 -18.339495  1.302268e+01   
252.200000  44    -19.666977  14.046066  -5.620911 -19.494763  1.437876e+01   
275.975000  45    -20.782041  15.428340  -5.353700 -20.629029  1.575069e+01   
299.750000  46    -21.878752  16.811600  -5.067153 -21.742143  1.711802e+01   
323.525000  47    -22.955992  18.180351  -4.775642 -22.833763  1.846333e+01   
347.300000  48    -24.012674  19.526206  -4.486468 -23.902415  1.978578e+01

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')